### 준비

In [1]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import os
import sys
import json

!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!pip install konlpy
!pip install soynlp

from pykospacing import Spacing
from konlpy.tag import Kkma, Komoran, Okt, Mecab
import soynlp

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
import re
import collections
import itertools
!pip install lda
import lda
import requests
import csv
import time
import math
import operator
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
from collections import defaultdict
from pandas import read_table
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-99purt2u
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-99purt2u
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 기본 전처리
* 참고: preprocessing 코드


In [ ]:
def preprocess_df(df):
  
    #basic
    df = df.iloc[:, 1:]
    df = df.dropna(subset=['user'])
    df = df.dropna(subset=['review'])
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)

    #options & help_count
    df["options"] = df["options"].fillna("")
    help = df[df.options.str.contains("명에게 도움이 되었습니다")]["options"].str.extract("(\d+)", expand = False)
    df.loc[help.index, "help_count"] = help
    df.loc[help.index, "options"] = ""
    help = df[df.options.str.contains("도움이 돼요")].index
    df.loc[help, "options"] = np.NaN
    help = df[df["options"] == ""].index
    df.loc[help, "options"] = np.NaN
    
    #scores
    df.total_score = df.total_score.str.extract("(\d\.\d)", expand = False).apply(lambda x: float(x))
    df.dur_score = df.dur_score.str.extract("(\d\.\d)", expand = False).apply(lambda x: float(x))
    df.price_score = df.price_score.str.extract("(\d\.\d)", expand = False).apply(lambda x: float(x))
    df.design_score = df.design_score.str.extract("(\d\.\d)", expand = False).apply(lambda x: float(x))
    df.delivery_score = df.delivery_score.str.extract("(\d\.\d)", expand = False).apply(lambda x: float(x))
    df.item_score = df.item_score.str.extract("(\d\.\d)", expand = False).apply(lambda x: float(x))

    #item_sale_price
    df["item_sale_price"] = df["item_sale_price"].apply(lambda x: int(x.split(",")[0] + x.split(",")[1]))
  
    return df

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/컨퍼런스/김다희/console/console_final.csv', encoding='utf-8')
df = preprocess_df(df)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 리뷰 전처리 - 단어 단위
* KOMORAN 형태소 분석기 이용
* 어근만 추출 vs __어느 정도 어미 살려서 추출__
* 불용어 처리: 형태소 분석 후 확인


In [ ]:
def load_stopwords():
    with open('/content/drive/Shareddrives/컨퍼런스/김다희/stopwords_temp.txt', 'r') as f:
        list_file = f.readlines()
    return list_file[0].split(",")

def extract_word(text):
    hangul = re.compile('[^가-힣]') 
    result = hangul.sub(' ', text) 
    return result

def make_wordlist(text, stopwords): #text = " ".join(data['Review'].tolist())
    print("리뷰들을 모아 분석하는 중입니다.....")
    #정규표현식 적용 -> 한글 표현만 남기기
    print("데이터 정제 중....")
    words = extract_word(text)
    #띄어쓰기 고치기 -> 꼬모란 할 필요 X
    #spacing = Spacing()
    #형태소 추출 -> 어근 살려서 
    print("형태소 추출 중....")
    kom = Komoran()
    words = kom.morphs(words)
    #한글자 제거 -> 유의미한 한 글자가 있을 수도 있으니 다시 확인
    print("한글자 제거 중....")
    words = [x for x in words if len(x)>1]
    #불용어 제거 -> 불용어 리스트 수정 필요
    print("불용어 제거 중....")
    words = [x for x in words if x not in stopwords]
    #최소횟수 미만 제거 -> 일단 5회로 지정 
    print("의미있는 단어리스트 생성 중....")
    time.sleep(1)
    minimum_count = 5
    final = []
    for i in tqdm(range(len(words))):
        tmp = words[i]
        if words.count(tmp) >= minimum_count:
            final.append(tmp)
    return set(final) #조건을 만족하는 단어 리스트

def preprocess_review(text, word_list):
    text = extract_word(text)
    kom = Komoran()
    text = kom.morphs(text)
    text = [x for x in text if x in word_list]
    return text

In [ ]:
#text = " ".join(df['review'].tolist()) 
text = " ".join(df['review'][:1000].tolist()) # 너무 오래걸려서 일단 일부만
stopwords = load_stopwords()
word_list = make_wordlist(text, stopwords)
final_review = preprocess_review(text, word_list)

리뷰들을 모아 분석하는 중입니다.....
데이터 정제 중....
형태소 추출 중....
한글자 제거 중....
불용어 제거 중....
의미있는 단어리스트 생성 중....


100%|██████████| 16045/16045 [00:06<00:00, 2620.85it/s]


### 리뷰 전처리 - 문장 단위
* WordRank용 전처리
* 형태소 분석기 수정 필요(komoran 오류로 일단 okt)

In [ ]:
# 1. 상품별 리뷰 생성 
name_list = df['item_name'].unique().tolist()
for i, name in enumerate(name_list):
  globals()['df{}'.format(i+1)] = df[df['item_name']==name].reset_index(drop=True)

# 2. 상품별 리뷰 전처리
from konlpy.tag import Okt, Kkma, Komoran

okt = Okt() 
kkma = Kkma()
kom = Komoran() # 왠지 모르겠는데 인코딩 오류남

def preprocess_review(review):
    total_review = ''
    #인풋리뷰
    for idx in range(len(review)):
        sentence = review[idx]
        #하나의 리뷰에서 문장 단위로 전처리
        #sentence = re.sub(name.split(' ')[0],'',r)
        #sentence = re.sub(name.split(' ')[1],'',sentence)
        sentence = re.sub('\n','',sentence)
        sentence = re.sub('\u200b','',sentence)
        sentence = re.sub('\xa0','',sentence)
        sentence = re.sub('([a-zA-Z])','',sentence)
        sentence = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',sentence)
        sentence = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',sentence)
        if len(sentence) == 0:
            continue
        sentence = okt.pos(sentence, stem=True)
        word = []
        for i in sentence:
            if not i[1] == 'Noun':
                continue
            if len(i[0]) == 1:
                continue
            word.append(i[0])
        word = ' '.join(word)
        word += '. '
        total_review += word
    return total_review.split('. ')

In [ ]:
# #상품 1개에 대해서 전처리
# review = df1['review']
# sentence_review = preprocess_review(review)
# sentence_review[:5]

#상품 전체에 대해서 전처리
sentence_review = []
for i in range(df['item_name'].nunique()):
      globals()['sentence_review{}'.format(i+1)] = preprocess_review(locals()['df{}'.format(i+1)]['review'])
      sentence_review.append(locals()['sentence_review{}'.format(i+1)])

### 워드 임베딩
* 참고: http://blog.skby.net/%EC%9B%8C%EB%93%9C-%EC%9E%84%EB%B2%A0%EB%94%A9word-embedding/
* 전처리된 단어들을 벡터화하여 계산 가능한 형태로 변환
* Frequency Based Embedding: BOW, Count Vector, __TF-IDF__
* Prediction Based Embedding: CBOW, Skip-gram, __Word2Vec__



In [ ]:
# 워드 임베딩 준비
review = df['review']
sentence_review = preprocess_review(review)
sentence_list = []
for sentence in sentence_review:
  sentence_list.append(list(sentence.split(' ')))

# 워드 임베딩
from gensim.models.word2vec import Word2Vec
model = Word2Vec(sentence_list)
model.init_sims(replace=True) # 메모리 상 불필요한 것 제거

In [ ]:
model.wv.most_similar('가격')

[('금액', 0.7191709280014038),
 ('품질', 0.6056124567985535),
 ('괜츈', 0.6014103889465332),
 ('퀄리티', 0.5717177391052246),
 ('가성', 0.5672529935836792),
 ('컨디션', 0.5657346248626709),
 ('퀄리', 0.563823401927948),
 ('이정', 0.5481315851211548),
 ('그닥', 0.5416619777679443),
 ('만해', 0.5398019552230835)]